# Linear regression 

In this exercise you will use linear regression to predict flat prices. One more time, training will be handled via gradient descent. Although, contratry to the first exercise, we will now:
* have multiple features (i.e. variables used to make the prediction),
* employ some basic feature engineering,
* work with a different loss function.

Let's start with getting the data.

In [1]:
%matplotlib inline

!wget -O mieszkania.csv https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
!wget -O mieszkania_test.csv https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?dl=1

--2017-03-07 12:24:20--  https://www.dropbox.com/s/zey0gx91pna8irj/mieszkania.csv?dl=1
Resolving www.dropbox.com... 162.125.66.1
Connecting to www.dropbox.com|162.125.66.1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dl.dropboxusercontent.com/content_link/Vx2dYQiFb2uLHoyy4y9BRjcEqTsvLM0ihBzeHv7q9SIBFdUeQtHwxmhOiFGwrT9Q/file?dl=1 [following]
--2017-03-07 12:24:21--  https://dl.dropboxusercontent.com/content_link/Vx2dYQiFb2uLHoyy4y9BRjcEqTsvLM0ihBzeHv7q9SIBFdUeQtHwxmhOiFGwrT9Q/file?dl=1
Resolving dl.dropboxusercontent.com... 162.125.66.6
Connecting to dl.dropboxusercontent.com|162.125.66.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6211 (6.1K) [text/csv]
Saving to: ‘mieszkania.csv’

mieszkania.csv      100%[===================>]   6.07K  --.-KB/s    in 0s      

2017-03-07 12:24:22 (152 MB/s) - ‘mieszkania.csv’ saved [6211/6211]

--2017-03-07 12:24:22--  https://www.dropbox.com/s/dbrj6sbxb4ayqjz/mieszkania_test.csv?

In [2]:
!head mieszkania.csv mieszkania_test.csv

==> mieszkania.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
104,mokotowo,2,2,1940,1,780094
43,ochotowo,1,1,1970,1,346912
128,grodziskowo,3,2,1916,1,523466
112,mokotowo,3,2,1920,1,830965
149,mokotowo,3,3,1977,0,1090479
80,ochotowo,2,2,1937,0,599060
58,ochotowo,2,1,1922,0,463639
23,ochotowo,1,1,1929,0,166785
40,mokotowo,1,1,1973,0,318849

==> mieszkania_test.csv <==
m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,cena
71,wolowo,2,2,1912,1,322227
45,mokotowo,1,1,1938,0,295878
38,mokotowo,1,1,1999,1,306530
70,ochotowo,2,2,1980,1,553641
136,mokotowo,3,2,1939,1,985348
128,wolowo,3,2,1983,1,695726
23,grodziskowo,1,1,1975,0,99751
117,mokotowo,3,2,1942,0,891261
65,ochotowo,2,1,2002,1,536499


In [1]:
import csv
from numpy import mean, dot
from math import log
from pprint import pprint

In [2]:
data = []
with open('mieszkania.csv') as f:
    reader = csv.DictReader(f)
    for line in reader:
        data.append(line)

Each row in the data represents a separate property. Our goal is to use the data from `mieszkania.csv` to create a model that can predict a property's price (i.e. `cena`) given its features (i.e. `m2,dzielnica,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny`). 

From now on, we should interfere only with `mieszkania.csv` (dubbed the training dataset) to make our decisions and create the model. The (only) purpose of `mieszkania_test.csv` is to test our model on **unseen** data.

Our predictions should minimize the so-called mean squared logarithmic error:
$$
MSLE = \frac{1}{n} \sum_{i=1}^n (\log(1+y_i) - \log(1+p_i))^2,
$$
where $y_i$ is the ground truth, and $p_i$ is our prediction.

Let's start with implementing the loss function.

In [13]:
def msle(ys, ps):
    assert len(ys) == len(ps)
    return sum((log(1 + y) - log(1 + p))**2 for (y,p) in zip(ys, ps)) / len(ys)
    # return sum((y - p)**2 for (y,p) in zip(ys, ps)) / len(ys)

The simplest model is predicting the same constant for each instance. Test your implementation of msle against outputing the mean price.

In [4]:
prices = [int(mieszkanie['cena']) for mieszkanie in data]
prices_m = mean(prices)

print msle(prices, [prices_m for _ in xrange(len(prices))])

0.391525353826


Recall that outputing the mean minimzes $MSE$. However, we're now dealing with $MSLE$.

Think of a constant that should result in the lowest $MSLE$.

In [5]:
#############################################
# TODO: Find this constant and compute msle #
#############################################

Now, let's implement a standard linear regression model. 

In [14]:
# 
a = 0
b = 0

# we have data prices
# linear regression: we may have more than one dimension
# each district will be a binary dimension
# take all districts and find unique values

districts = set(m['dzielnica'] for m in data)

# now we want to make our data into a matrix, or list of lists with schema
# m2,ilość_sypialni,ilość_łazienek,rok_budowy,parking_podziemny,'grodziskowo', 'ochotowo', 'wolowo', 'mokotowo'

matrix = [
    [
        int(m['m2']), 
        int(m['ilość_sypialni']), 
        int(m['ilość_łazienek']), 
        int(m['rok_budowy']),  # TODO maybe scale
        int(m['parking_podziemny']),
        1 if m['dzielnica'] == 'grodziskowo' else 0,
        1 if m['dzielnica'] == 'ochotowo' else 0,
        1 if m['dzielnica'] == 'wolowo' else 0,
        1 if m['dzielnica'] == 'mokotowo' else 0,
        1  # for the constant
    ] for m in data]

# pprint(matrix)

parameters = [0 for _ in xrange(len(matrix[0]))]
alfa = 0.01

"""
Update the i-th parameter.
"""
def update(i):
    return parameters[i] - alfa / len(matrix) * sum((dot(parameters, vector) - price) * vector[i] for (vector, price) in zip(matrix, prices))

# print update(0) / len(matrix)

msle_log = []
for _ in xrange(100):
    error = msle(prices, [dot(parameters, vector) for vector in matrix])
    print parameters
    print error
    msle_log.append(error)
#     print [dot(parameters, vector) for vector in matrix[:5]]
    parameters = [update(i) for i in xrange(len(parameters))]
    
    
    # Now want happens is we get an error
    # notebooks suck when it comes to debugging
    # but are potentially nice for plotting
    # maybe I can use iPython and plot from pycharm too
    # somehow we get predicted price which is negative
    # p price must be positive
    # how do we debug that?
    
#     There is an error in the update function
# No idea how to debug it though
# No use thinking now, try later
# Maybe write corrent code in Pycharm for a change

# Another idea: compute the slope numerically

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
331792609839
[499954.52180000005, 11830.43455, 9537.936450000001, 9980354.67145, 2576.22815, 753.53485, 1574.0124, 1192.71235, 1558.9353, 5079.1949]
3.84939359787e+20
[-15733531418.658009, -390873477.5410344, -318267503.45426124, -384908003060.64294, -97071895.744385839, -45961904.148415886, -52967252.226534843, -48960585.04739365, -48280058.469987519, -196169799.89233193]
5.72149978173e+29
[606531133052818.5, 15068555998272.529, 12269570099782.141, 14839380943247098.0, 3742428581615.7686, 1772000491940.6453, 2042103051326.438, 1887517558957.218, 1861326609883.3164, 7562947712107.6201]
8.5040824825e+38
[-2.3383627048381526e+19, -5.8093884528685952e+17, -4.7302939287344282e+17, -5.7210345808410293e+20, -1.4428205207534061e+17, -68316031813897304.0, -78729310076146416.0, -72769566140279184.0, -71759825200190208.0, -2.9157473323051328e+17]
1.26399408596e+48
[9.0151024138989402e+23, 2.2396966799248242e+22, 1.8236727829809003e+22, 2.2056335637538685e+25, 5.562

/Users/piotr/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: overflow encountered in double_scalars


[-1.3267623095705619e+166, -3.2961856708507618e+164, -2.6839188312695719e+164, -3.2460546167515144e+167, -8.1864112973910115e+163, -3.8761795149974409e+163, -4.4670179289369635e+163, -4.1288683504717411e+163, -4.0715767156500274e+163, -1.6543642510056175e+164]
inf
[5.1150739254615596e+170, 1.270780248792715e+169, 1.034732682173176e+169, 1.2514531963260251e+172, 3.1561115859510867e+168, 1.4943855899847737e+168, 1.7221718440487219e+168, 1.5918048537267617e+168, 1.5697171786918507e+168, 6.3780794664521111e+168]
inf
[-1.9720172237486398e+175, -4.8992459830239623e+173, -3.9892105196446503e+173, -4.8247342928626782e+176, -1.2167774108966683e+173, -5.7613128672540699e+172, -6.6394984475470396e+172, -6.1368938829414925e+172, -6.0517391496255382e+172, -2.4589444347368146e+173]
inf
[7.6027286945034354e+179, 1.8888089600841476e+178, 1.5379624944890018e+178, 1.86008242777789e+181, 4.6910485492934249e+177, 2.2211620733525352e+177, 2.5597294362531914e+177, 2.3659600259611625e+177, 2.3331302754570915

/Users/piotr/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:38: RuntimeWarning: overflow encountered in double_scalars
/Users/piotr/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:38: RuntimeWarning: invalid value encountered in double_scalars


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]
nan
[nan, nan,

Note that the loss function that the algorithms optimizes (i.e $MSE$) differs from $MSLE$. We've already seen that this may result in a suboptimal solution.

How can you change the setting so that we optimze $MSLE$ instead?

Hint: 
Be lazy. We don't want to change algorithm.

In [ ]:
#############################################
# TODO: Optimize msle and compare the error #
#############################################

Without any feature engineering our model approximates the price as a linear combination of original features:
$$
\text{price} \approx w_1 \cdot \text{area} + w_2 \cdot \text{district} + \dots.
$$
Let's now introduce some interactions between the variables. For instance, let's consider a following formula:
$$
\text{price} \approx w_1 \cdot \text{area} \cdot \text{avg. price in the district per sq. meter} + w_2 \cdot \dots + \dots.
$$
Here, we model the price with far greater granularity, and we may expect to see more acurate results.

Add some feature engineering to your model. Be sure to play with the data and not with the algorithm's code. 

Think how to make sure that your model is capable of capturing the $w_1 \cdot \text{area} \cdot \text{avg. price...}$ part, without actually computing the averages.

Hint: 
<sub><sup><sub><sup><sub><sup>
Is having a binary encoding for each district and multiplying it by area enough?
</sup></sub></sup></sub></sup></sub>

Hint 2: 
<sub><sup><sub><sup><sub><sup>
Why not multiply everything together? I.e. (A,B,C) -> (AB,AC,BC).
</sup></sub></sup></sub></sup></sub>

In [ ]:
###############################################
# TODO: Implement the feature engieering part #
###############################################

In [ ]:
##############################################################
# TODO: Test your solution on the training and test datasets #
##############################################################